# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [12]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
from sympy import *
from sympy import solve

from PIL import Image

# Provided functions

In [13]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [14]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [15]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [16]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR IMPLEMENTATION>
    
    second_moment = np.dot(lines, lines.T)
    w, v = np.linalg.eig(second_moment)
    min_idx = np.argmin(w)
    vp = v[:, min_idx]
    vp = vp / vp[-1]
    return vp

In [17]:
def get_horizon_line(vpts):
    """
    Calculates the ground horizon line.
    """
    # <YOUR IMPLEMENTATION>
    line1 = vpts[:, 0]
    line2 = vpts[:, 1]
    
    # Line equation: line[0] * x + line[1] * y + line[2] = 0
    horizon_line = np.cross(line1, line2)
    #Normalizing to magnitude 1
    horizon_line /= np.sqrt(horizon_line[0]**2 + horizon_line[1]**2)
    
    return horizon_line

In [28]:
def plot_horizon_line(im, horizon_line):
    """
    Plots the horizon line.
    """
    # <YOUR IMPLEMENTATION>
    x_range = im.shape[1]
    x = np.arange(x_range)
    y = (- horizon_line[2] - horizon_line[0] * x) / horizon_line[1]
    
    plt.figure()
    plt.imshow(im)
    plt.plot(x, y)    
    plt.show()

In [19]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR IMPLEMENTATION>
    vpts1 = vpts[:, 0][:, np.newaxis]  # converting to column vectors
    vpts2 = vpts[:, 1][:, np.newaxis]
    vpts3 = vpts[:, 2][:, np.newaxis]
    
    f, px, py= symbols('f, px, py')
    K_inv_T = Matrix([[1/f, 0, 0], [0, 1/f, 0], [-px/f, -py/f, 1]])
    K_inv = Matrix([[1/f, 0, -px/f], [0, 1/f, -py/f], [0, 0, 1]])
    
    eq1 = vpts1.T * K_inv_T * K_inv * vpts2
    eq2 = vpts1.T * K_inv_T * K_inv * vpts3
    eq3 = vpts2.T * K_inv_T * K_inv * vpts3
    
    f, px, py = solve([eq1[0], eq2[0], eq3[0]], (f, px, py))[0]

    return abs(f), px, py

In [20]:
def get_rotation_matrix(vpts, f, u, v):
    """
    Computes the rotation matrix using the camera parameters.
    """
    # <YOUR IMPLEMENTATION>
    Z = vpts[:, 0][:, np.newaxis]  # convertig to column vectors
    X = vpts[:, 1][:, np.newaxis]
    Y = vpts[:, 2][:, np.newaxis]

    K = np.array([[f, 0, u], [0, f, v], [0, 0, 1]]).astype(np.float)
    K_inv = np.linalg.inv(K)

    r1 = K_inv.dot(X)
    r2 = K_inv.dot(Y)
    r3 = K_inv.dot(Z)

    #normalization to magnitude 1
    r1 = r1 / np.linalg.norm(r1)
    r2 = r2 / np.linalg.norm(r2)
    r3 = r3 / np.linalg.norm(r3)

    R = np.concatenate((r1, r2, r3), axis=1)
       
    return R

In [26]:
def estimate_height(im, person_coord, obj_coord, horizon_line):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    # <YOUR IMPLEMENTATION>
    H = 1.6764  # 5ft 6in in meters
#     H = 1.8288  # 6 ft in meters
    
    vpz = vpts[:, 2]  # Vertical vp.
    # Person as reference.
    t_person = person_coord[:, 0]
    b_person = person_coord[:, 1]

    # Obj coords.
    r = obj_coord[:, 0]
    b = obj_coord[:, 1]

    line_b_person_b = np.cross(b_person, b)
    v = np.cross(line_b_person_b, horizon_line)
    v = v / v[-1]
    
    line_v_t_person = np.cross(v, t_person)
    line_r_b = np.cross(r, b)
    t = np.cross(line_v_t_person, line_r_b)
    t = t / t[-1]
    
    height = H*(np.linalg.norm(r-b) * np.linalg.norm(vpz-t) /
              np.linalg.norm(t-b) / np.linalg.norm(vpz-r))
    
    return height

# Main function

In [ ]:
im = np.asarray(Image.open('CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])
print(vpts)

In [22]:
im = np.asarray(Image.open('CSL.jpg'))
num_vpts = 3
vpts = np.zeros((3, num_vpts))
lines1 = np.array([[4.22587523e+01, 9.86037554e+00, -1.26776257e+01], [5.01470528e+02, 4.97244652e+02, 4.90201527e+02], [-9.81789385e+04, -1.06443143e+05, -1.12022941e+05]])
vpts[:, 0] = get_vanishing_point(lines1)


lines2 = np.array([[-2.57001550e+01, -1.05824168e+01, 1.81412859e+01],[1.28500775e+02, 1.28500775e+02, 1.28500775e+02], [5.29360684e+03, -1.54770948e+04, -5.10867708e+04]])
vpts[:, 1] = get_vanishing_point(lines2)

lines3 = np.array([[-1.80702110e+02, -1.15371347e+02, -7.50608766e+01], [-6.95008117e+00, -2.78003247e+00, 0.00000000e+00], [ 1.50920865e+05, 1.11295916e+05, 2.48691055e+04]])
vpts[:, 2] = get_vanishing_point(lines3)

print(vpts)

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
# plot_horizon_line(im, horizon_line)

[[-2.96469945e+02  1.28080923e+03  1.55304240e+02]
 [ 2.20520969e+02  2.19212527e+02  2.15434587e+04]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00]]


In [23]:
num_vpts = 3
vpts = np.zeros((3, num_vpts))
lines1 = np.array([[4.22587523e+01, 9.86037554e+00, -1.26776257e+01], [5.01470528e+02, 4.97244652e+02, 4.90201527e+02], [-9.81789385e+04, -1.06443143e+05, -1.12022941e+05]])
vpts[:, 0] = get_vanishing_point(lines1)


lines2 = np.array([[-2.57001550e+01, -1.05824168e+01, 1.81412859e+01],[1.28500775e+02, 1.28500775e+02, 1.28500775e+02], [5.29360684e+03, -1.54770948e+04, -5.10867708e+04]])
vpts[:, 1] = get_vanishing_point(lines2)

lines3 = np.array([[-1.80702110e+02, -1.15371347e+02, -7.50608766e+01], [-6.95008117e+00, -2.78003247e+00, 0.00000000e+00], [ 1.50920865e+05, 1.11295916e+05, 2.48691055e+04]])
vpts[:, 2] = get_vanishing_point(lines3)

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)
print(f, u, v)

704.061358073542 137.634665651078 243.433240018998


In [24]:
# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(vpts, f, u, v)
print(R)

[[ 8.51330465e-01  8.29103367e-04 -5.24629156e-01]
 [-1.80373423e-02  9.99453804e-01 -2.76902042e-02]
 [ 5.24319648e-01  3.30364301e-02  8.50880427e-01]]


In [27]:
# Part 4
im = np.asarray(Image.open('CSL.jpg'))
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'spike statue', 'lamp posts')
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(im, coords['person'], coords[obj], horizon_line)
    print(obj, height)

Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of spike statue
Click on the bottom coordinate of spike statue
Click on the top coordinate of lamp posts
Click on the bottom coordinate of lamp posts
Estimating height of CSL building
CSL building 23.072659298501254
Estimating height of spike statue
spike statue 10.738230630692003
Estimating height of lamp posts
lamp posts 5.568722676958392
